# Parseo

Basciamente tengo los xml en una carpeta, quiero extraer la informacion dejarlo  
en diccionarios para que quede listo para crear un grafo

Tambien poner en minusculas las letras y quitar acentos etc.

In [1]:
import html
import xml.etree.ElementTree as ET
import shutil
import os

def check_derogados(atribs, file):
    os.makedirs("derogados", exist_ok=True)
    if "derogado" not in atribs:
        print("error! no hay info de derogacion")
    else:
        # mover los derogados
        derogado = atribs["derogado"]   
        if "no" in derogado:
            return 0
        else:
            shutil.move(file.path, f"derogados/{file.name}")
            return 1

def parse_xml(xml_root):
    """
    Recibe un root de un XML de ley chilena
    y devuelve un diccionario con:
      - id_ley, tipo, numero, derogado
      - lista de parrafos (cada uno con id, texto limpio, y tipo)
    """
    
    # Usamos este namespace asi podemos acceder a los tipos de elementos facilmente
    ns = {"n": "http://www.leychile.cl/esquemas"}

    id_ley = xml_root.attrib.get("normaId")
    derogado = xml_root.attrib.get("derogado", "desconocido")
    
    tipo = numero = None
    tipo_num = xml_root.find(".//n:TiposNumeros/n:TipoNumero", ns)
    if tipo_num is not None:
        tipo = tipo_num.findtext("n:Tipo", default="", namespaces=ns)
        numero = tipo_num.findtext("n:Numero", default="", namespaces=ns)

    organismo = None
    organismo_type = xml_root.find(".//n:Organismos/n:Organismo", ns)
    if organismo_type is not None and organismo_type.text:
        organismo = organismo_type.text

    ley_data = {
        "id_ley": id_ley,
        "tipo": tipo,
        "numero": numero,
        "organismo":organismo,
        # "derogado": derogado,
        "modified_to": [],
        "parrafos": []
    }

    # encavezado
    encabezado = xml_root.find(".//n:Encabezado/n:Texto", ns)
    if encabezado is not None and encabezado.text:
        ley_data["parrafos"].append({
            # "id": "encabezado",
            "tipo": "Encabezado",
            "texto": encabezado.text.strip()
        })

    # Estructuras funcnioanles 
    for e in xml_root.findall(".//n:EstructuraFuncional", ns):
        texto = e.findtext("n:Texto", default="", namespaces=ns).strip()
        tipo_parte = e.attrib.get("tipoParte", "Desconocido")
        id_parte = e.attrib.get("idParte")
        derogado_parte = e.attrib.get("derogado", "no derogado")
        if derogado_parte =="no derogado":

            ley_data["parrafos"].append({
                "id": id_parte,
                "tipo": tipo_parte,
                "texto": texto,
                "derogado": derogado_parte
            })
        else:
            continue

    
    # MEtadatos mas adelante quizsa
    
    # Promulgacion
    # promulgacion = xml_root.find(".//n:Promulgacion/n:Texto", ns)
    # if promulgacion is not None and promulgacion.text:
    #     ley_data["parrafos"].append({
    #         "id": "promulgacion",
    #         "tipo": "Promulgacion",
    #         "texto": ""
    #         # "texto": promulgacion.text.strip()
    #     })

    return ley_data


def parsing_files(path_law, debug=False,is_derogado=False):
    law = dict()
    with os.scandir(path_law) as directory:
        for file in directory:
            name = file.name
            law_code = name.split(".")[0]
            try :
                tree = ET.parse(file)
            except:
                print(f"error : {law_code}")
                continue

            root = tree.getroot()
            atribs = root.attrib

            if is_derogado:
                derr = check_derogados(atribs,file)
                if derr== 1:
                    print(f"derogado {name}")
                else:
                    print(f"no derogado {name}")
            
            if debug:
                print(f"Parseando {name}")
            data = parse_xml(root)
            law[law_code] = data

            

    return law

path_law = "law_chile"

law = parsing_files(path_law)



In [2]:
def norm_text(text, remove_accents=True, keep_refs=True):
    text = html.unescape(text)
    text = text.lower()
    
    if remove_accents:
        text = re.sub(
            r'[áàäâÁÀÄÂ]', 'a',
            re.sub(r'[éèëêÉÈËÊ]', 'e',
            re.sub(r'[íìïîÍÌÏÎ]', 'i',
            re.sub(r'[óòöôÓÒÖÔ]', 'o',
            re.sub(r'[úùüûÚÙÜÛ]', 'u', text)))))
    
    # Espacios y saltos
    text = re.sub(r'\s+', ' ', text).strip()

    # Normalizar comillas y guiones
    text = text.replace("“", '"').replace("”", '"').replace("–", "-")

    # Unificar formato de artículos (opcional)
    # text = re.sub(r'\bArt[íi]?culo\s+(\w+):', r'Articulo \1:', text)


    return text


In [4]:
import html
import re
for doc in law.values():
    doc["organismo"] = norm_text(doc["organismo"])
    for parr in doc["parrafos"]:
        parr["texto"] = norm_text(parr["texto"])

# 
Añadimos las relaciones y guardamos  el resultado den parsed_law.json

In [5]:
import json


with open("data/relations.json", "r", encoding="utf-8") as f:
    relations = json.load(f)


for doc in law.values():
    id_law = str(doc["id_ley"])
    
    # No hay acceso a todas las leyes de all_law
    # pues
    # 1) no considero las derogadas
    # 2) hay algunas que no se pueden obtener por xml pues 
    # estan fallando en el BCN por alguna razon de ellos
    try:
        # Obtenemos todas las leyes que modifica la id_ley
        modified_to = relations[id_law]
        doc["modified_to"] = modified_to

    except KeyError:
        continue  

with open("data/parsed_law.json", "w", encoding="utf-8") as f:
    json.dump(law, f, ensure_ascii=False, indent=4)


#### Todos los tipos de parrafos que tenemos

In [ ]:
types = set()
for doc in law.values():
    for parr in doc["parrafos"]:
        types.add(parr["tipo"])


print(types)


{'Disposición Transitoria', 'Capítulo', 'Encabezado', 'Enumeración', 'Artículo Transitorio', 'Doble Articulado', 'Otros', 'Párrafo', 'Título', 'Libro', 'Artículo', 'Parágrafo'}


#### Todos las leyes que tenemos asociadas a su code

In [ ]:
misc = []
numbers = []
for doc in law.values():
    number = doc["numero"]
    id_law = doc["id_ley"]
    # print(f"Ley número {number} con id {id_law}")
    
    if number.isnumeric():
        numbers.append(int(number  ) )
    else:
        if number == "S/N":
            number = 999_991
        elif number == "ORGÁNICO DE TRIBUNALES":
            number = 999_992
            doc["id"] = number
        
        misc.append(number)

# print(sorted(numbers))
print(misc)

[999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, '5984 EXENTA', 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 999991, 